[//]: # (Image References)

[image1]: learning_curves.png "Learning Curves for Proximal Policy Optimization with Generalized Advantage Estimation"

# Report: Continuous Control Project

This work trains simulated robotic arms to track a dynamic target position for their end effector. The environment has a continuous state space, representing orientations and rates of arm joints, as well as a continuous action space, representing torques applied to arm joints. For training, this work uses [proximal policy optimization (PPO)](https://arxiv.org/pdf/1707.06347.pdf) in an actor-critic form and a [generalized advantage estimate (GAE)](https://arxiv.org/pdf/1506.02438.pdf). Two deep neural networks are trained: one for the stochastic policy that maps the robots state to actions, and one for the value function that estimates the expected returns from a given state of the robot. 

## Learning Algorithm

As previosly mentioned, this work uses PPO for policy training and GAE for advantage estimation. 

(DISCUSS PPO SURROGATE FUNCTION AND VALUE LOSS FUNCTION)

### Execution Notebook: `Continuous_Control.ipynb`

### Source Code and Model Architecture 

### Data Files

### Hyperparameters

## Plot of Rewards

![PPO-GAE Learning Curves][image1]

## Ideas for Future Work